<h1><center>Laboratorio 7: La desperación de Mr. Lepin 🐼</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla, Ignacio Meza
- Auxiliar: Sebastián Tinoco
- Ayudante: Felipe Arias, Diego Cortez

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Luis Gutiérrez 
- Nombre de alumno 2: Juan Carlos Cuevas


### **Link de repositorio de GitHub:** `https://github.com/LuisGutierrezG/LuisGutierrezG-LABS`

### Indice 

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripción del laboratorio](#Descripción-del-laboratorio.)
4. [Desarrollo](#Desarrollo)

# Temas a tratar

- Aplicar Pandas para obtener características de un DataFrame.
- Aplicar Pipelines.
- Aplicar Clusters sobre un conjunto de datos.

## Reglas:

- **Grupos de 2 personas**
- Asistencia **obligatoria** a instrucciones del lab (viernes 16.15). Luego, pueden quedarse trabajando en las salas o irse.
- **No se revisarán entregas de personas ausentes**. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio

- Comprender y aprovechar las ventajas que nos ofrece la librería `pandas` con respecto a trabajar en Python 'puro'.
- Crear nuevas características para entrenar un modelo de clustering.
- Comprender como aplicar pipelines de Scikit-Learn para generar procesos más limpios.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `numpy`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre arreglos (*o tensores*).

## Descripción del laboratorio.

### Importamos librerias utiles 😸

In [4]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
from datetime import date

today = date.today()
print(today)

from IPython.display import HTML

# Libreria para plotear (En colab esta desactualizado plotly)
# !pip install --upgrade plotly
import plotly.express as px
import plotly.graph_objects as go

# Librerias utiles
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split 
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

2023-06-05


In [5]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except: 
    print('Ignorando conexión drive-colab')

Ignorando conexión drive-colab


# Segmentación de Clientes en Tienda de Retail 🛍️

<p align="center">
  <img width=300 src="https://s1.eestatic.com/2018/04/14/social/la_jungla_-_social_299733421_73842361_854x640.jpg">
</p>

## 1.1 Cargar Dataset

Mr. Lepin, en una nueva reunión, le cuenta a ud y su equipo que los resultados derivados del análisis exploratorio de dato presentaron una gran utilidad para la empresa y que tiene un gran entusiasmo por continuar trabajando con ustedes.
Es por esto, que Mr. Lepin les pide que cargue y visualicen algunas de las filas que componen el Dataset.
A continuación un extracto de lo parlamentado en la reunión:

    - Usted: Es un gran logro para nuestro equipo que usted haya encontrado excelente el EDA. ¿Qué tiene en mente ahora?
    - Mr. Lepin: Resulta que hace algún tiempo, mientras tomaba un mojito en una reunión de gerentes en Panamá, oí a un *chato* acerca de **LRMFP**, que es un modelo que permite personificar a los clientes a través de la fabricación de distintos atributos que describen a los clientes. Lo encontré es-tu-pendo ñatito. 
    - Usted: Ehh bueno. Investigaremos acerca de este modelo y veremos lo que podemos hacer.

Por ende, su siguiente tarea es calcular **LRMFP** sobre cada cliente y luego hacer un análisis de las características generadas. Para esto, el área de ventas les entrega un nuevo archivo llamado `online_retail_II_cleaned.pickle`, quien posee los datos del DataFrame original limpios y listos para obtener las características solicitadas por Mr. Lepin.

In [6]:
df_retail = pd.read_pickle("online_retail_II_cleaned.pickle")
df_retail = df_retail.astype(
    {
        "Invoice": "category",
        "StockCode": "category",
        "Description": "category",
        "Description": str,
        "Customer ID": "category",
        "Country": "category"
    }
)
df_retail.head()


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [7]:
df_retail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400947 entries, 0 to 400946
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      400947 non-null  category      
 1   StockCode    400947 non-null  category      
 2   Description  400947 non-null  object        
 3   Quantity     400947 non-null  int64         
 4   InvoiceDate  400947 non-null  datetime64[ns]
 5   Price        400947 non-null  float64       
 6   Customer ID  400947 non-null  category      
 7   Country      400947 non-null  category      
dtypes: category(4), datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 15.9+ MB


In [8]:
# n uniques customers
df_retail["Customer ID"].nunique()

4314

## 1.2 Creación de nuevas Caracteristicas [2 Puntos] 

Como ya se les comento, Mr. Lepin esta interesado en obtener las características **LRMFP**, para esto les señala que estas características se construyen en base a las siguientes definiciones:

- **Length (L)**: Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.

- **Recency (R)**: Indica hace cuanto tiempo el cliente realizo su ultima compra. Notar que para este caso, mientras mas grande es el valor, menos interes posee el usuario para repetir una compra en uno de los locales.

- **Monetary (M)**: El término "monetario" se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.

- **Frequency (F)**: Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente. 

- **Periodicity (P)**: Representa si los clientes visitan las tiendas con regularidad.

$$Periodicity(n)=std(IVT_1, ..., IVT_n)$$

&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Donde $IVT$ denota el tiempo entre visitas y n representa el número de valores de tiempo entre visitas de un cliente.
 

$$IVT_i=date\_diff(t_{i+1},t)$$

En base a las definiciones señaladas, diseñe una función que permita obtener las características **LRMFP** recibiendo un DataFrame como entrada. Para esto, no estará permitido el uso de iteradores, utilice todas las herramientas que les ofrece `pandas` para realizar esto.

Una referencia que le puede ser útil es el [documento original](https://www.researchgate.net/publication/315979555_LRFMP_model_for_customer_segmentation_in_the_grocery_retail_industry_a_case_study) en donde se propone este método.

**Ejemplo de Resultado Esperado:**

| Customer ID | Length | Recency | Frequency | Monetary | Periodicity |
|------------:|-------:|--------:|----------:|---------:|------------:|
|   12346.0   |    294 |      67 |        46 |   -64.68 |        37.0 |
|   12347.0   |     37 |       3 |        71 |  1323.32 |         0.0 |
|   12349.0   |    327 |      43 |       107 |  2646.99 |        78.0 |
|   12352.0   |     16 |      11 |        18 |   343.80 |         0.0 |
|   12356.0   |     44 |      16 |        84 |  3562.25 |        12.0 |

**Respuesta:**

In [9]:
# group by customer id y date unique
#df1 = df_retail.groupby(['Customer ID', 'InvoiceDate']).count()
# se crea un shift para ver la diferencia de dias entre compras
#df1['shift'] = df1.groupby(level=0)['Invoice'].shift(1)
#df1

In [10]:
def customer_features(df):
    # Clientes unicos
#    unique_customers = df_retail["Customer ID"].unique()
#    unique_customers

    # Columna Length:
    df_length = df.groupby('Customer ID')['InvoiceDate'].apply(lambda x: x.max() - x.min()).reset_index()
    df_length.columns = ['Customer ID', 'Lenght']
    df_length['Lenght'] = df_length['Lenght'].dt.days    # se convierte la columna 'Lenght' a días

    # Columna Recency:
    max_date = max(df_retail["InvoiceDate"])
    df_recency = df.groupby('Customer ID')['InvoiceDate'].apply(lambda x: max_date - x.max()).reset_index()
    df_recency.columns = ['Customer ID', 'Recency']
    df_recency['Recency'] = df_recency['Recency'].dt.days    # se convierte la columna 'Recency' a días

    # Columna Frequency:
    df_frequency = df.groupby('Customer ID')['Invoice'].count().reset_index()
    df_frequency.columns = ['Customer ID', 'Frequency']

    # Columna Monetary:
    # Primero agrupamos por 'Customer ID' y 'Invoice' para obtener el precio de cada compra
        #  df_boleta = df.groupby(['Customer ID', 'Invoice'])['Price'].sum().reset_index()
        #  df_monetary = df_boleta.groupby('Customer ID')['Price'].avg().reset_index()
    df1 = df.groupby('Customer ID')['Price'].sum().reset_index()
    df2 = df.groupby('Customer ID')['Invoice'].nunique().reset_index()

    df_monetary = df1.merge(df2, on='Customer ID', how='inner')
    df_monetary['Monetary'] = df_monetary['Price'] / df_monetary['Invoice']
    df_monetary = df_monetary.drop(['Price', 'Invoice'], axis=1)

    # cantidad de invoices por cliente
#    df_invoices = df.groupby('Customer ID')['Invoice'].nunique().reset_index()

#    df_monetary.columns = ['Customer ID', 'Monetary']


    # Columna Periodicity:
    df_periodicity = df.groupby('Customer ID')['InvoiceDate'].apply(lambda x: x.max() - x.min()).reset_index()
    df_periodicity.columns = ['Customer ID', 'Periodicity']
    df_periodicity['Periodicity'] = df_periodicity['Periodicity'].dt.days    # se convierte la columna 'Periodicity' a días

    # Unimos las features:
    df_features = df_length.merge(df_recency, on='Customer ID', how='inner')
    df_features = df_features.merge(df_frequency, on='Customer ID', how='inner')
    df_features = df_features.merge(df_monetary, on='Customer ID', how='inner')
    df_features = df_features.merge(df_periodicity, on='Customer ID', how='inner')

    return df_features


In [11]:
df_features = customer_features(df_retail)
df_features.head()

,Customer ID,Lenght,Recency,Frequency,Monetary,Periodicity
0,12346.0,196,164,33,18.760,196
1,12347.0,37,2,71,81.475,37
2,12348.0,0,73,20,14.390,0
3,12349.0,181,42,102,291.780,181
4,12351.0,0,10,21,49.460,0


**Resúesta de Pauta:**

## 1.3 Pipelines 👷

Finalmente *Don Mora* le pregunta si seria posible realizar un pipeline para realizar una segmentación de los clientes con los nuevos datos generados, a lo que usted responde que **sí** y propone la utilización de k-means para la segmentación.

A continuación siga los pasos requeridos para obtener la segmentación de clientes.

### 1.3.1 Estandarizar Caracteristicas [0.5 puntos]

Construya una clase llamada ``MinMax()`` utilizando ``BaseEstimator`` y ``TransformerMixin`` para realizar una transformación de cada una de las columnas de un DataFrame utilizando ``ColumnTransformer()`` más tarde (tome como referencia el siguiente [enlace](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer)).


 Para esto considere que Min-Max escaler queda dada por la ecuación:

$$MinMax = \dfrac{x-min(x)}{max(x) - min(x)}$$

Con esto buscamos que los valores que componen a las columnas se muevan en el rango de valores $[0, 1]$.

**Respuesta:**

In [12]:
df_features

,Customer ID,Lenght,Recency,Frequency,Monetary,Periodicity
0,12346.0,196,164,33,18.7600,196
1,12347.0,37,2,71,81.4750,37
2,12348.0,0,73,20,14.3900,0
3,12349.0,181,42,102,291.7800,181
4,12351.0,0,10,21,49.4600,0
...,...,...,...,...,...,...
4309,18283.0,275,17,217,79.5450,275
4310,18284.0,0,66,28,91.0900,0
4311,18285.0,0,295,12,100.2000,0
4312,18286.0,247,111,67,143.1500,247


In [46]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler


In [47]:
class MinMax(BaseEstimator, TransformerMixin):
    def __init__(self, columns_to_scale):
        self.columns_to_scale = columns_to_scale
        self.ct = ColumnTransformer(
            transformers=[
                ('MinMax', MinMaxScaler(), columns_to_scale)
            ], remainder='passthrough'  # Pasar las columnas no seleccionadas sin cambios
        )

    def fit(self, X, y=None):
        self.ct.fit(X, y)
        return self

    def transform(self, X):
        return self.ct.transform(X)
        

In [48]:
columns_to_scale = ['Lenght', 'Recency', 'Frequency', 'Monetary', 'Periodicity']
scaler = MinMax(columns_to_scale)

In [49]:
scaler.fit(df_features.iloc[:, 1:])
df_transformed = scaler.transform(df_features.iloc[:, 1:])
df_transformed = pd.DataFrame(df_transformed, columns=df_features.iloc[:, 1:].columns)
df_transformed


,Lenght,Recency,Frequency,Monetary,Periodicity
0,0.525469,0.439678,0.005748,0.001713,0.525469
1,0.099196,0.005362,0.012574,0.007438,0.099196
2,0.000000,0.195710,0.003413,0.001314,0.000000
3,0.485255,0.112601,0.018143,0.026638,0.485255
4,0.000000,0.026810,0.003593,0.004515,0.000000
...,...,...,...,...,...
4309,0.737265,0.045576,0.038800,0.007262,0.737265
4310,0.000000,0.176944,0.004850,0.008316,0.000000
4311,0.000000,0.790885,0.001976,0.009148,0.000000
4312,0.662198,0.297587,0.011856,0.013069,0.662198


### 1.3.2 `T-SNE` Pipeline [1.0 puntos]

Para comenzar introduciéndose en el uso de pipeline, decide probar realizando un pipeline enfocado en la reducción de dimensionalidad y así hacer no decepcionar a Mr. Lepin con la clusterización del modelo. 

Configure un pipeline utilizando el algoritmo `T-SNE` sobre los datos **LRMFP**, donde, para la realización del pipeline considera los siguientes pasos:

1. Como primer paso obtenga las características **LRMFP** desde el DataFrame ``df_retail_II_cleaned.pickle`` utilizando la función ``custom_features`` creada anteriormente, junto a ``FunctionTransformer()``. Considere esto como el primer paso de su pipeline.
2. En segundo lugar usando ``ColumnTransformer()`` aplique el MinxMax scaler creado por usted sobre todas las columnas generadas en el paso anterior. 
3. Finalmente, aplique un último paso donde obtiene las 2 componentes más relevantes utilizando el algoritmo `T-SNE` de sckit-learn.

Tras aplicar las transformaciones sobre el dataset **LRMFP**, gráfique las componentes obtenidas en la reducción de dimensionalidad.

**Respuesta:**

In [50]:
# se crea un pipeline para aplicar el scaler y el tsne
pipeline = Pipeline(
    steps=[
        ('scaler', scaler),
        ('tsne', TSNE(n_components=2, random_state=42))

    ]
)

df_tsne = pipeline.fit_transform(df_features.iloc[:, 1:])
df_tsne = pd.DataFrame(df_tsne, columns=['x', 'y'])
df_tsne.head()



C:\Users\luchi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\manifold\_t_sne.py:795: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
C:\Users\luchi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\manifold\_t_sne.py:805: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


,x,y
0,13.948992,-7.859164
1,-18.700863,-1.009477
2,41.571354,-56.952805
3,37.110031,14.203368
4,-50.034061,11.923832


In [51]:
# visualizacion de la data
fig = px.scatter(df_tsne, x='x', y='y')
fig.show()


### 1.3.3 Clustering

#### 1.3.3.1 Método del Codo [1 puntos]

Utilizando la clase creada para escalamiento, aplique el método del codo para visualizar cual es el número de clusters que mejor se ajustan a los datos. Realice esto utilizando el algoritmo K-means dentro de un pipeline para un $k \in [1,20]$, donde k representa el número de clusters del k-means. Para la realización de esta sección y la próxima (1.3.3.2), considere los mismos pasos utilizados para el t-sne, pero **permutando el algoritmo de reducción de dimensionalidad por k-means.**

A través del grafico obtenido, comente y justifique que valor de k escogería para realizar el k-means.

**Respuesta:**

In [58]:
# se itera entre 1 y 20 para ver cual es el mejor numero de clusters
inertia = []
for i in range(1, 20):
    kmeans = KMeans(n_clusters=i, random_state=42)
    kmeans.fit(df_transformed)
    inertia.append(kmeans.inertia_)

fig = px.line(x=range(1, 20), y=inertia)
fig.show()



In [59]:
# se ve que el mejor numero de clusters es 4
pipeline = Pipeline(
    steps=[
        ('scaler', scaler),
        ('kmeans', KMeans(n_clusters=4, random_state=42))

    ]
)

pipeline.fit(df_features.iloc[:, 1:])
df_features['cluster'] = pipeline.predict(df_features.iloc[:, 1:])
df_features.head()



,Customer ID,Lenght,Recency,Frequency,Monetary,Periodicity,cluster
0,12346.0,196,164,33,18.760,196,2
1,12347.0,37,2,71,81.475,37,2
2,12348.0,0,73,20,14.390,0,2
3,12349.0,181,42,102,291.780,181,2
4,12351.0,0,10,21,49.460,0,2


In [60]:
# se visualiza la data agrupada por cluster
fig = px.scatter(df_tsne, x='x', y='y', color=df_features['cluster'])
fig.show()


#### 1.3.3.2 Segmentación de Clientes con K-Means 🎁 [1 punto]

En base a la elección de k realizada en la sección anterior, utilice este valor escogido y entrene un modelo de K-means utilizando el mismo pipeline de scikit-learn utilizado anteriormente.

Una vez ajustado los datos, genere una tabla con los promedios (o medianas) para cada uno de los atributos, agrupando estos por el clúster que pertenecen. ¿Es posible observar agrupaciones coherentes?, ¿Qué tipo de clientes posee el retail?, Justifique su respuesta y no decepcione a Mr. Lepin.


**Respuesta:**

In [62]:
# se agrupa por cluster y se ve el promedio de cada feature y la cantidad de clientes por cluster
df_features.groupby('cluster').agg({'Lenght': 'mean', 'Recency': 'mean', 'Frequency': 'mean', 'Monetary': 'mean', 'Periodicity': 'mean', 'Customer ID': 'count'})


,Lenght,Recency,Frequency,Monetary,Periodicity,Customer ID
cluster,,,,,,
0,338.817618,17.967742,239.810174,66.473197,338.817618,806
1,239.238217,47.989809,113.836943,72.332368,239.238217,785
2,57.452216,62.857448,52.192205,72.276765,57.452216,1873
3,10.947059,258.276471,24.167059,85.090477,10.947059,850


**Respuesta Esperada:**

|         | Length  | Recency   | Frequency | Monetary | Periodicity |       |
|---------|---------|-----------|----------|-------------|-------|-------|
| Cluster |         |           |          |             |       |       |
|    0    |   258.8 |      45.2 |     76.1 |      1107.7 | 107.6 |   449 |
|    1    |    76.1 |     217.6 |     45.5 |       791.7 |  14.1 |   466 |
|    2    |   368.5 |       4.8 |   2715.0 |    226621.6 |   4.2 |     4 |
|    3    |    85.3 |      45.7 |     65.8 |      1047.0 |  10.5 |   987 |
|    4    |   347.2 |      15.9 |   1658.0 |     35829.3 |   8.0 |    25 |
|    5    |   298.0 |      29.8 |    183.8 |      3639.9 |  32.0 |  1188 |

#### 1.3.3.3 Plot de K-Means 📈 [0.5 puntos]



Por último, Mr. Lepin, impaciente de no entender lo que usted intenta explicarle, le solicita que por favor muestre algún resultado "visual" de los grupos encontrados.

Para esto, grafique nuevamente las características encontradas usando `T-SNE` (no calcule de nuevo, simplemente utilice las proyecciones encontradas) y agregue las labels calculadas con kmeans como el argumento `color`.   

Comente: ¿Se separan bien los distintos clusters en la visualización?


**Respuesta:**

In [63]:
# se visualiza la data agrupada por cluster
fig = px.scatter(df_tsne, x='x', y='y', color=df_features['cluster'])
fig.show()


# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por correo, Discord o U-cursos.

![Gracias Totales!](https://i.pinimg.com/originals/65/ae/27/65ae270df87c3c4adcea997e48f60852.gif "bruno")


<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>